In [1]:
import numpy as np
from snnpy import *
import grispy as gsp
from time import time
from tqdm import tqdm 
import gc 

### (varying sizes, fixed dimension)

In [2]:
dims = [3]
index_sizes = np.logspace(3, 5, num=7, dtype=int)  
radius = [0.05, 0.1, 0.15, 0.2, 0.25]
query_size = 1000

In [3]:
snn_index_time = np.zeros((len(index_sizes), len(dims)))
snn_query_time = np.zeros((len(index_sizes), len(dims)))
gp_index_time = np.zeros((len(index_sizes), len(dims)))
gp_query_time = np.zeros((len(index_sizes), len(dims)))

In [4]:
for i in range(len(index_sizes)):
    print("index size: ", index_sizes[i])
    isize = index_sizes[i]
    for j in tqdm(range(len(dims))):
        n_dim = dims[j]
        data = np.random.uniform(size=(isize + query_size, n_dim))  
        I = data[:isize]
        Q = data[isize:]
        
        inst = time()
        snn_model = build_snn_model(I)  
        snn_index_time[i,j] = time() - inst
        
        inst = time()
        grid = gsp.GriSPy(I)
        gp_index_time[i,j] = time() - inst

        for q in range(query_size):
            qst = time()
            for r in range(len(radius)):
                snnind = snn_model.query_radius(Q[q], radius[r], return_distance=False)
            snn_query_time[i,j] += (time() - qst) / len(radius)
            
            qst = time()
            for r in range(len(radius)):
                gpdist, gpind = grid.bubble_neighbors(Q[[q]], distance_upper_bound=radius[r])
            gp_query_time[i,j] += (time() - qst) / len(radius)
        
        snn_query_time[i,j] = snn_query_time[i,j] / query_size
        gp_query_time[i,j] = gp_query_time[i,j] / query_size
        
        del snn_model, grid
        gc.collect()

index size:  1000


100%|██████████| 1/1 [01:47<00:00, 107.22s/it]


index size:  2154


100%|██████████| 1/1 [01:29<00:00, 89.27s/it]


index size:  4641


...

100%|██████████| 1/1 [03:10<00:00, 190.91s/it]


index size:  46415


100%|██████████| 1/1 [03:19<00:00, 199.16s/it]


index size:  100000


100%|██████████| 1/1 [02:41<00:00, 161.29s/it]


In [5]:
with open('result/add2_snn_index_time.npy', 'wb') as f:
    np.save(f, snn_index_time)
    
with open('result/add2_snn_query_time.npy', 'wb') as f:
    np.save(f, snn_query_time)
    
with open('result/add2_gp_index_time.npy', 'wb') as f:
    np.save(f, gp_index_time)

with open('result/add2_gp_query_time.npy', 'wb') as f:
    np.save(f, gp_query_time)